In [10]:
from qiskit.circuit import QuantumCircuit, QuantumRegister
import ffsim
import math
import numpy as np
from passive_extended_matchgate_simulator.raw_estimation import raw_estimate
from passive_extended_matchgate_simulator.utils import ucj_to_compatible, ucj_to_compatible_fully_reduced, calculate_trajectory_count, extract_circuit_data

norb = 6
nelec = (3, 3)
alpha_alpha_indices = [(p, p + 1) for p in range(norb - 1)]
alpha_beta_indices = [(p, p) for p in range(norb)]
qubits = QuantumRegister(2 * norb)
circuit = QuantumCircuit(qubits)
ucj_op = ffsim.random.random_ucj_op_spin_balanced(norb, 
                                                  interaction_pairs=(alpha_alpha_indices, alpha_beta_indices),
                                                  with_final_orbital_rotation=False,
                                                  diag_coulomb_mean=0, 
                                                  diag_coulomb_scale=math.sqrt(.1), 
                                                  diag_coulomb_normal=True)
circuit.append(ffsim.qiskit.PrepareHartreeFockJW(norb, nelec), qubits)
circuit.append(ffsim.qiskit.UCJOpSpinBalancedJW(ucj_op), qubits)

statevec = ffsim.qiskit.final_state_vector(circuit)
ffsim_probs = np.abs(statevec.vec)**2
bitstrings = ffsim.addresses_to_strings(range(len(ffsim_probs)), norb, nelec)
ffsim_p_and_b = list(zip(ffsim_probs, bitstrings))

epsilon = .1
delta = .01
p = 1

compatible = ucj_to_compatible(circuit)
compatible.draw(fold=-1)

┌───┐┌──────────────┐                                                                                                                                                         ┌──────────────┐
 q_0: ┤ X ├┤0             ├─■───────────────────────────■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤0             ├
      ├───┤│              │ │P(-0.48074)                │                                                                                                                           │              │
 q_1: ┤ X ├┤1             ├─■─────────────■─────────────┼───────────────────────────■───────────────────────────────────────────────────────────────────────────────────────────────┤1             ├
      ├───┤│              │               │P(-0.41848)  │                           │                                                                                               │              │
 q_2: ┤ X ├┤2             ├───────────────■─────────────┼─────────────■─────────────┼───────────────────────────■───────────────────────────────────────────────────────────────────┤2             ├
      └───┘│              │                             │             │P(0.32377)   │                           │                                                                   │              │
 q_3: ─────┤3             ├─────────────────────────────┼─────────────■─────────────┼─────────────■─────────────┼─────────────────────────■─────────────────────────────────────────┤3             ├
           │              │                             │                           │             │P(0.059617)  │                         │                                         │              │
 q_4: ─────┤4             ├─────────────────────────────┼───────────────────────────┼─────────────■─────────────┼────────────■────────────┼──────────────■──────────────────────────┤4             ├
           │              │                             │                           │                           │            │P(0.22046)  │              │                          │              │
 q_5: ─────┤5             ├─────────────────────────────┼───────────────────────────┼───────────────────────────┼────────────■────────────┼──────────────┼─────────────■────────────┤5             ├
      ┌───┐│   Orb_rot_jw │                             │P(-0.13346)                │                           │                         │              │             │            │   Orb_rot_jw │
 q_6: ┤ X ├┤6             ├─■───────────────────────────■───────────────────────────┼───────────────────────────┼─────────────────────────┼──────────────┼─────────────┼────────────┤6             ├
      ├───┤│              │ │P(-0.48074)                                            │P(0.034025)                │                         │              │             │            │              │
 q_7: ┤ X ├┤7             ├─■─────────────■─────────────────────────────────────────■───────────────────────────┼─────────────────────────┼──────────────┼─────────────┼────────────┤7             ├
      ├───┤│              │               │P(-0.41848)                                                          │P(0.23295)               │              │             │            │              │
 q_8: ┤ X ├┤8             ├───────────────■─────────────■───────────────────────────────────────────────────────■─────────────────────────┼──────────────┼─────────────┼────────────┤8             ├
      └───┘│              │                             │P(0.32377)                                                                       │P(-0.045059)  │             │            │              │
 q_9: ─────┤9             ├─────────────────────────────■─────────────■───────────────────────────────────────────────────────────────────■──────────────┼─────────────┼────────────┤9             ├
           │              │                                       

In [11]:
print(len(ffsim_p_and_b))
ffsim_p_and_b[:10]

400


[(np.float64(0.8152359040253669), 455),
 (np.float64(0.000283841028285084), 711),
 (np.float64(0.0007342649610105777), 839),
 (np.float64(0.02228969603132299), 903),
 (np.float64(0.011326860782089328), 1223),
 (np.float64(0.0038971288528834827), 1351),
 (np.float64(0.02742668979852275), 1415),
 (np.float64(0.0005400003714952237), 1607),
 (np.float64(0.005144073487380353), 1671),
 (np.float64(7.517627574767936e-06), 1799)]

In [12]:
import time
raw_estimate_values = []
t1 = time.time()
for b in bitstrings[:10]:
    raw_estimate_values.append(raw_estimate(circuit=compatible,outcome_state=b, epsilon=epsilon, delta=delta, p=p))
t2 = time.time()
print(f"took {t2 - t1} seconds")
raw_estimate_values

took 0.32570886611938477 seconds


[0.8055337773848495,
 0.0002688651427671688,
 0.0007712769446783543,
 0.02232511873735701,
 0.011367509566705996,
 0.003943880614907552,
 0.027091528207017275,
 0.000548694733156841,
 0.004716623113710125,
 5.55291587129192e-06]

In [13]:
diff = np.abs(ffsim_probs[:10] - raw_estimate_values)
print("max difference", max(diff))
data = extract_circuit_data(compatible)
extent = data[0]
t = calculate_trajectory_count(epsilon, delta, extent, p)
print("trajectory count", t)
diff


max difference 0.009702126640517394
trajectory count 69952


array([9.70212664e-03, 1.49758855e-05, 3.70119837e-05, 3.54227060e-05,
       4.06487846e-05, 4.67517620e-05, 3.35161592e-04, 8.69436166e-06,
       4.27450374e-04, 1.96471170e-06])